In [1]:
import numpy as np
from flask import Flask,render_template,request

In [2]:
app = Flask(__name__)
import pickle as pkl
X = pkl.load(open("X.pkl","rb"))
tfidf = pkl.load(open("model.pkl","rb"))

In [3]:
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
nltk.download('punkt')

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


stopwords = stopwords.words('english')
porter = PorterStemmer()

[nltk_data] Downloading package punkt to /Users/ishan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ishan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def preprocess(text):
  # removing punctuations
    import string
    punc = string.punctuation
    for c in text:
      if c in punc:
        text = text.replace(c,"")
    # print(text)

    # tokenization
    words = word_tokenize(text)
    # print("After tokenization: {}".format(words))

    # stop words removal
    clean_words=[]
    for word in words:
      if word not in stopwords:
        clean_words.append(word)

    words = clean_words
    # print("After stop words removal: {}".format(words))

    # stemming
    stemmed_words = []
    for word in words:
      stemmed_words.append(porter.stem(word))

    words = stemmed_words
    # print("After stemming: {}".format(words))

    # converting to lowercase
    lowercase_words = []
    for word in words:
      lowercase_words.append(word.lower())

    words = lowercase_words
    # print("After converting to lowecase: {}".format(words))

    #adding to processed docs
    processed_text = ' '.join(map(str, words))

    return processed_text

In [5]:
from sklearn.metrics.pairwise import cosine_similarity


def searchQuery(k, query):
  processed_query = preprocess(query)
  query_vec = tfidf.transform([processed_query])

  cosine_values = cosine_similarity(X,query_vec)
  cosine_values = cosine_values.reshape((-1,))

  results = []
  cosine_sorted_index = cosine_values.argsort()[-k:][::-1]
  # print(cosine_sorted_index)
  for i in cosine_sorted_index:
    results.append(('p3_d{}.txt'.format(i+1),cosine_values[i]))

  return results

In [6]:
query = "the simpsons american sitcom"

results = searchQuery(10,query)
content = []
for result in results:
  if (result[1] != 0):
    with open(result[0],'r',encoding='utf-8') as file:
        text = file.read()
        text = str(text)
    content.append(text)

results

[('p3_d9.txt', 0.5397861890380348),
 ('p3_d13.txt', 0.0343666631954557),
 ('p3_d14.txt', 0.015905595269496132),
 ('p3_d1.txt', 0.00387021003280606),
 ('p3_d15.txt', 0.0),
 ('p3_d12.txt', 0.0),
 ('p3_d11.txt', 0.0),
 ('p3_d10.txt', 0.0),
 ('p3_d8.txt', 0.0),
 ('p3_d7.txt', 0.0)]

In [7]:
@app.route('/')
def home():
  return render_template('index.html')

In [8]:
@app.route('/search',methods=['POST'])
def search():
  query = str(request.form.values())
  results = searchQuery(10,query)
  content = []
  for result in results:
    if (result[1] != 0):
      with open(result[0],'r',encoding='utf-8') as file:
          text = file.read()
          text = str(text)
      content.append(text)
  return render_template('index.html',doc_names=content)

In [13]:
if __name__ == "__main__":
  app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 48] Address already in use

In [14]:
from multiprocessing import Process

server = Process(target=app.run)
server.start()
# ...
server.terminate()
server.join()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


Process Process-1:
Traceback (most recent call last):
  File "/Users/ishan/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/ishan/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ishan/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 990, in run
    run_simple(host, port, self, **options)
  File "/Users/ishan/opt/anaconda3/lib/python3.7/site-packages/werkzeug/serving.py", line 1030, in run_simple
    s.bind(server_address)
OSError: [Errno 48] Address already in use
